In [8]:
import numpy as np
import pandas as pd
from CodingFeatures import DNAFeatureExtraction, stop_delimited_fragments, codon_lookup

%load_ext autoreload
%autoreload 1
%aimport CodingFeatures

In [9]:
with open('../data/tmp_seq_rpoB-20kb.txt','r') as f:
    seq_raw = f.read()
print(seq_raw[:50])

CATTACTGTCACAATTTCCAACAGCACCAAATTACCCCCAGGCGGATGGT


In [10]:
codon_table, c_lookup = codon_lookup()

In [11]:
seq_object = DNAFeatureExtraction(seq_raw).decode_dna(c_lookup)